In [74]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Hourly
import numpy as np

df = pd.read_csv("bicikelj_train.csv")

def prepare_data(df, weather=False):

    df["timestamp"] = [pd.to_datetime(ts).tz_localize(None) for ts in df["timestamp"].values]


    df["day_of_week"] = [x.day_of_week for x in df.timestamp]
    df["pon"] = df["day_of_week"] == 0
    df["pet"] = df["day_of_week"] == 4
    df["vikend"] = df["day_of_week"].isin([5,6])
    df = df.drop(['day_of_week'], axis=1)


    df["time_int"] = [int(ts.hour * 60 + ts.minute) for ts in df["timestamp"]]


    pocitnice = ["26. 6. 2022", "27. 6. 2022", "28. 6. 2022", "29. 6. 2022", "30. 6. 2022", "1. 7. 2022", "2. 7. 2022", "3. 7. 2022", "4. 7. 2022", "5. 7. 2022", "6. 7. 2022", "7. 7. 2022", "8. 7. 2022", "9. 7. 2022", "10. 7. 2022", "11. 7. 2022", "12. 7. 2022", "13. 7. 2022", "14. 7. 2022", "15. 7. 2022", "16. 7. 2022", "17. 7. 2022", "18. 7. 2022", "19. 7. 2022", "20. 7. 2022", "21. 7. 2022", "22. 7. 2022", "23. 7. 2022", "24. 7. 2022", "25. 7. 2022", "26. 7. 2022", "27. 7. 2022", "28. 7. 2022", "29. 7. 2022", "30. 7. 2022", "31. 7. 2022", "1. 8. 2022", "2. 8. 2022", "3. 8. 2022", "4. 8. 2022", "5. 8. 2022", "6. 8. 2022", "7. 8. 2022", "8. 8. 2022", "9. 8. 2022", "10. 8. 2022", "11. 8. 2022", "12. 8. 2022", "13. 8. 2022", "14. 8. 2022", "15. 8. 2022", "16. 8. 2022", "17. 8. 2022", "18. 8. 2022", "19. 8. 2022", "20. 8. 2022", "21. 8. 2022", "22. 8. 2022", "23. 8. 2022", "24. 8. 2022", "25. 8. 2022", "26. 8. 2022", "27. 8. 2022", "28. 8. 2022", "29. 8. 2022", "30. 8. 2022", "31. 8. 2022"] + ["31. 10. 2022", "1. 11. 2022", "2. 11. 2022", "3. 11. 2022", "4. 11. 2022", "25. 12. 2022", "26. 12. 2022", "27. 12. 2022", "28. 12. 2022", "29. 12. 2022", "30. 12. 2022", "31. 12. 2022", "1. 1. 2023", "2. 1. 2023", "6. 2. 2023", "7. 2. 2023", "8. 2. 2023", "9. 2. 2023", "10. 2. 2023", "10. 4. 2023", "26. 4. 2023", "27. 4. 2023", "28. 4. 2023", "29. 4. 2023", "30. 4. 2023", "1. 5. 2023", "2. 5. 2023", "25. 6. 2023"]
    pocitnice = [datetime.strptime(x,"%d. %m. %Y").date() for x in pocitnice]
    df["date"] = [ts.date() for ts in df["timestamp"]]
    df["pocitnice"] = df["date"].isin(pocitnice)
    df = df.drop(['date'], axis=1)
    
    if weather:

        # Set time period
        start = datetime(2022, 1, 1)
        end = datetime(2022, 12, 31)

        # Create Point for Vancouver, BC
        location = Point(46.056946, 14.505751, 70)

        # Get daily data for 2018
        data = Hourly(location, start, end)
        data = data.fetch()

        # # Plot line chart including average, minimum and maximum temperature
        # data.plot(y=['tavg', 'tmin', 'tmax'])
        # plt.show()
        data = data.reset_index()
        data["timestamp_round"] = data["time"]
        # data = data.drop(['time', 'snow', 'tsun'], axis=1)
        data = data[["temp","dwpt","rhum","prcp","timestamp_round"]]

        def hour_rounder(t):
            # Rounds to nearest hour by adding a timedelta hour if minute >= 30
            return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
                    +timedelta(hours=t.minute//30))

        df["timestamp_round"] = [hour_rounder(ts) for ts in df["timestamp"]]

        df = df.merge(data, on='timestamp_round', how='left')
        df = df.drop(['timestamp_round'], axis=1)
    return df

df = prepare_data(df,weather=True)
df

,timestamp,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,POGAČARJEV TRG-TRŽNICA,KONGRESNI TRG-ŠUBIČEVA ULICA,CANKARJEVA UL.-NAMA,BREG,GRUDNOVO NABREŽJE-KARLOVŠKA C.,MIKLOŠIČEV PARK,BAVARSKI DVOR,TRG OF-KOLODVORSKA UL.,...,SUPERNOVA LJUBLJANA - RUDNIK,pon,pet,vikend,time_int,pocitnice,temp,dwpt,rhum,prcp
0,2022-08-02 13:04:00,17,17,20,26,10,10,16,7,12,...,1,False,False,False,784,True,28.2,15.9,47.0,0.0
1,2022-08-02 13:10:00,18,17,19,25,10,11,15,7,12,...,1,False,False,False,790,True,28.2,15.9,47.0,0.0
2,2022-08-02 13:14:00,17,18,19,25,9,11,16,7,12,...,1,False,False,False,794,True,28.2,15.9,47.0,0.0
3,2022-08-02 13:20:00,18,15,20,26,12,11,18,6,13,...,1,False,False,False,800,True,28.2,15.9,47.0,0.0
4,2022-08-02 13:25:00,20,16,19,23,12,10,17,6,15,...,1,False,False,False,805,True,28.2,15.9,47.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7734,2022-10-01 18:03:00,18,17,14,7,7,5,10,20,25,...,7,False,False,True,1083,False,15.6,12.2,80.0,0.0
7735,2022-10-01 18:07:00,20,17,13,7,7,5,11,19,24,...,7,False,False,True,1087,False,15.6,12.2,80.0,0.0
7736,2022-10-01 18:13:00,19,18,14,5,8,3,10,20,23,...,7,False,False,True,1093,False,15.6,12.2,80.0,0.0
7737,2022-10-01 18:16:00,19,17,16,4,6,3,9,17,21,...,7,False,False,True,1096,False,15.6,12.2,80.0,0.0


In [75]:
dt = pd.read_csv("bicikelj_test.csv")
dt = prepare_data(dt,weather=True)
dt

,timestamp,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,POGAČARJEV TRG-TRŽNICA,KONGRESNI TRG-ŠUBIČEVA ULICA,CANKARJEVA UL.-NAMA,BREG,GRUDNOVO NABREŽJE-KARLOVŠKA C.,MIKLOŠIČEV PARK,BAVARSKI DVOR,TRG OF-KOLODVORSKA UL.,...,SUPERNOVA LJUBLJANA - RUDNIK,pon,pet,vikend,time_int,pocitnice,temp,dwpt,rhum,prcp
0,2022-08-04 23:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,1404,True,22.0,17.2,74.0,0.0
1,2022-08-05 00:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,True,False,24,True,23.9,16.8,64.0,0.0
2,2022-08-08 04:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,False,False,298,True,19.8,14.9,73.0,0.0
3,2022-08-08 05:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,False,False,358,True,23.4,13.1,52.0,0.0
4,2022-08-11 21:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,1273,True,24.6,9.9,39.0,0.0
5,2022-08-11 22:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,1333,True,20.4,12.7,61.0,0.0
6,2022-08-16 07:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,430,True,23.0,18.0,73.0,0.0
7,2022-08-16 08:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,490,True,25.9,16.8,57.0,0.0
8,2022-08-18 17:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,1050,True,25.5,18.5,65.0,0.0
9,2022-08-18 18:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,False,False,False,1107,True,25.5,18.5,65.0,0.0


In [76]:
from datetime import timedelta
def opremi_z_urami(df, legal_values, tol = 10):
    l= [[],[]]
    for t in [1,2]:
        for index, row in df.iterrows():
                target = row['timestamp'] - timedelta(hours=t)
                out = None
                targetminus = target
                targetplus = target
                for i in range(tol):
                    targetminus -= timedelta(minutes=1)
                    targetplus += timedelta(minutes=1)
                    if targetminus in legal_values:
                        out = targetminus
                        break
                    if targetplus in legal_values:
                        out = targetplus
                        break
                l[t-1].append(out)
    df["timestampminus1"] = l[0]
    df["timestampminus2"] = l[1]
    return df
df = opremi_z_urami(df,legal_values = df["timestamp"].values)
dt = opremi_z_urami(dt,legal_values = df["timestamp"].values, tol=120) # sam da zafilamo

In [10]:
dt

,timestamp,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,POGAČARJEV TRG-TRŽNICA,KONGRESNI TRG-ŠUBIČEVA ULICA,CANKARJEVA UL.-NAMA,BREG,GRUDNOVO NABREŽJE-KARLOVŠKA C.,MIKLOŠIČEV PARK,BAVARSKI DVOR,TRG OF-KOLODVORSKA UL.,...,LEK - VEROVŠKOVA,VOKA - SLOVENČEVA,SUPERNOVA LJUBLJANA - RUDNIK,pon,pet,vikend,time_int,pocitnice,timestampminus1,timestampminus2
0,2022-08-04 23:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,1404,True,2022-08-04 22:19:00,2022-08-04 21:29:00
1,2022-08-05 00:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,True,False,24,True,2022-08-04 22:19:00,2022-08-04 22:19:00
2,2022-08-08 04:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,False,False,298,True,2022-08-08 03:53:00,2022-08-08 02:53:00
3,2022-08-08 05:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,False,False,358,True,2022-08-08 03:53:00,2022-08-08 03:53:00
4,2022-08-11 21:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,1273,True,2022-08-11 20:08:00,2022-08-11 19:08:00
5,2022-08-11 22:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,1333,True,2022-08-11 20:08:00,2022-08-11 20:08:00
6,2022-08-16 07:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,430,True,2022-08-16 06:06:00,2022-08-16 05:06:00
7,2022-08-16 08:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,490,True,2022-08-16 06:06:00,2022-08-16 06:06:00
8,2022-08-18 17:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,1050,True,2022-08-18 16:25:00,2022-08-18 15:29:00
9,2022-08-18 18:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,False,False,1107,True,2022-08-18 16:25:00,2022-08-18 16:25:00


In [4]:
df.columns[-11:]

Index(['ROŠKA - STRELIŠKA', 'LEK - VEROVŠKOVA', 'VOKA - SLOVENČEVA',
       'SUPERNOVA LJUBLJANA - RUDNIK', 'pon', 'pet', 'vikend', 'time_int',
       'pocitnice', 'timestampminus1', 'timestampminus2'],
      dtype='object')

In [67]:
from sklearn import preprocessing
def get_pp(df,sourse_of_history, postaja="PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE"):
    attrs = [postaja] + [x for x in list(df.columns) if x.islower()]
    for x in ["date", "time", "timestamp_round","day_of_week"]:
        if x in attrs: attrs.remove(x)

    dg = df[attrs]
    dg.columns = ["y"] + list(dg.columns)[1:]
    
    dj = sourse_of_history[attrs]
    dj.columns = dg.columns    
    
    dh = dj[["y","timestamp"]]
    dh.columns = ["pred_eno_uro","timestampminus1"]
    
    to_use = dg[~np.isnan(dg["timestampminus1"]) & ~np.isnan(dg["timestampminus2"])]
    
    pp = pd.merge(to_use, dh, on="timestampminus1",
                    how = 'left')
    
    dh.columns = ["pred_dvema_urama","timestampminus2"]
    pp = pd.merge(pp, dh, on="timestampminus2",
                    how = 'left')
    pp = pp.drop(["timestamp","timestampminus1","timestampminus2"],axis=1)
    X = pp.iloc[:,1:]
    #X  = preprocessing.normalize(X)
    y = np.array(pp.iloc[:,0])
    return (pp,np.array(X),y)

pp,_,_ = get_pp(df,df, "PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE")
pp

,y,pon,pet,vikend,time_int,pocitnice,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,coco,pred_eno_uro,pred_dvema_urama
0,16,False,False,False,895,True,29.8,15.3,41.0,0.0,50.0,10.8,16.7,1016.0,3.0,17,17
1,19,False,False,False,900,True,29.8,15.3,41.0,0.0,50.0,10.8,16.7,1016.0,3.0,17,17
2,16,False,False,False,905,True,29.8,15.3,41.0,0.0,50.0,10.8,16.7,1016.0,3.0,18,17
3,16,False,False,False,910,True,29.8,15.3,41.0,0.0,50.0,10.8,16.7,1016.0,3.0,18,17
4,19,False,False,False,915,True,29.8,15.3,41.0,0.0,50.0,10.8,16.7,1016.0,3.0,19,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7283,18,False,False,True,1083,False,15.6,12.2,80.0,0.0,233.0,6.8,11.1,1014.4,3.0,20,19
7284,20,False,False,True,1087,False,15.6,12.2,80.0,0.0,233.0,6.8,11.1,1014.4,3.0,20,19
7285,19,False,False,True,1093,False,15.6,12.2,80.0,0.0,233.0,6.8,11.1,1014.4,3.0,20,19
7286,19,False,False,True,1096,False,15.6,12.2,80.0,0.0,233.0,6.8,11.1,1014.4,3.0,20,18


In [68]:
import numpy as np
from sklearn import preprocessing
X = np.array(pp.iloc[:,1:])
#X  = preprocessing.normalize(X)
y = np.array(pp.iloc[:,0])

In [125]:
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_score
# model = LinearRegression()
# model.fit(X, y)
# scores = cross_val_score(model, X, y, scoring='r2', cv=5) # TODO scoring?
# print(scores)
# #r_sq = model.score(Xt, yt)
# #print(f"coefficient of determination: {r_sq}")

[ 0.4772207   0.42923846  0.2247545  -0.15379843  0.16235668]


In [44]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [55]:
from sklearn import linear_model
from sklearn import metrics

digreg = linear_model.LogisticRegression(max_iter=1000)

digreg.fit(X_train, y_train)

y_pred = digreg.predict(X_test)
print(y_pred)

print("MSE is:",
metrics.mean_squared_error(y_test, y_pred))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def gbc(X_train,y_train,X_test):
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred
    # #cross_val_score(clf, X_train, y_train, cv=5)
    # print(y_pred)
    # mse = mean_squared_error(y_test,y_pred)
    # print(mse)
#clf.score(X_test, y_test)

In [13]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def gbr(X_train,y_train,X_test):
    # clf = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
    clf = GradientBoostingRegressor(random_state=0).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred
    # #cross_val_score(clf, X_train, y_train, cv=5)
    # print(y_pred)
    # mse = mean_squared_error(y_test,y_pred)
    # print(mse)
#clf.score(X_test, y_test)

In [77]:
def run(df,dt,inside=True,round=True,weather=False):
    postaje = [x for x in list(df.columns) if x.isupper()]
    for postaja in postaje:
        print(postaja)
        if inside:
            _,X,y = get_pp(df,df,postaja)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)
        else:
            _,X_train,y_train = get_pp(df,df,postaja)
            _,X_test,y_test = get_pp(dt,df,postaja)
        y_pred = gbr(X_train,y_train,X_test)
        if inside:
            mse = mean_squared_error(y_test,y_pred)
            print(mse)
        else:
            if round:
                dt[postaja] = np.round(y_pred)
            else:
                dt[postaja] = y_pred
        
    return dt

res = run(df,dt,inside=False,weather=True)
        
        

PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE
POGAČARJEV TRG-TRŽNICA
KONGRESNI TRG-ŠUBIČEVA ULICA
CANKARJEVA UL.-NAMA
BREG
GRUDNOVO NABREŽJE-KARLOVŠKA C.
MIKLOŠIČEV PARK
BAVARSKI DVOR
TRG OF-KOLODVORSKA UL.
MASARYKOVA DDC
VILHARJEVA CESTA
PARK NAVJE-ŽELEZNA CESTA
TRG MDB
PARKIRIŠČE NUK 2-FF
AMBROŽEV TRG
GH ŠENTPETER-NJEGOŠEVA C.
ILIRSKA ULICA
TRŽAŠKA C.-ILIRIJA
TIVOLI
STARA CERKEV
KINO ŠIŠKA
ŠPICA
BARJANSKA C.-CENTER STAREJŠIH TRNOVO
ZALOŠKA C.-GRABLOVIČEVA C.
TRŽNICA MOSTE
ROŽNA DOLINA-ŠKRABČEVA UL.
DUNAJSKA C.-PS PETROL
PLEČNIKOV STADION
DUNAJSKA C.-PS MERCATOR
LIDL - VOJKOVA CESTA
ŠPORTNI CENTER STOŽICE
KOPRSKA ULICA
MERCATOR CENTER ŠIŠKA
CITYPARK
BTC CITY/DVORANA A
BTC CITY ATLANTIS
TRNOVO
P+R BARJE
P + R DOLGI MOST
BONIFACIJA
ANTONOV TRG
BRATOVŠEVA PLOŠČAD
BS4-STOŽICE
SAVSKO NASELJE 2-LINHARTOVA CESTA
SAVSKO NASELJE 1-ŠMARTINSKA CESTA
SITULA
ŠTEPANJSKO NASELJE 1-JAKČEVA ULICA
HOFER-KAJUHOVA
BRODARJEV TRG
PREGLOV TRG
LIDL-LITIJSKA CESTA
ŽIVALSKI VRT
CESTA NA ROŽNIK
ŠMARTINSKI PARK
POLJANSKA-P

In [34]:
get_pp(dt,df)[0].iloc[:,[0,-2,-2]]

,y,pred_eno_uro,pred_eno_uro
0,2.245731,11,11
1,0.231799,11,11
2,0.338076,0,0
3,0.817280,0,0
4,6.850781,4,4
5,2.853217,4,4
6,2.095725,0,0
7,3.180993,0,0
8,3.234416,1,1
9,3.745413,1,1


In [78]:
#"temp","dwpt","rhum","prcp","wdir","wspd","wpgt","pres","coco"
# res.drop(["pon",	"pet",	"vikend",	"time_int",	"pocitnice",	"timestampminus1",	"timestampminus2"],axis=1).to_csv(index=False,path_or_buf='/home/lema/Documents/UOZP/bicikelj/lala_reg_round_vreme.csv')
res.drop(["pon",	"pet",	"vikend",	"time_int",	"pocitnice",	"timestampminus1",	"timestampminus2", "temp","dwpt","rhum","prcp"],axis=1).to_csv(index=False,path_or_buf='/home/lema/Documents/UOZP/bicikelj/lala_reg_round_vreme4.csv')

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

def svm(X_train,y_train,X_test):
    svm_model_linear = SVC(kernel = 'linear', C = 1)
    svm_model_linear.fit(X_train, y_train) # Training the model using fit method
    y_pred = svm_model_linear.predict(X_test)# model accuracy for X_test. Score method takes X_test and predicts the output and calculates the accuracy by comparing the predicted outputs with y_test.
    return(y_pred)

y_pred= svm(X_train,y_train,X_test)
mse = mean_squared_error(y_test,y_pred)
mse

76.35802469135803